In [10]:
from data.gen_data import build_X, build_Y, build_XY
from data.gen_data_config import gen_data_config
from evaluation import score
import pandas as pd
import numpy as np
from util import plot_roi, categori_reverse
from model import build_model, build_embed_model
from keras.optimizers import SGD, Adam, RMSprop, Adagrad
import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold, datasets

In [11]:
def tsne_trans(input):
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=501)
    return tsne.fit_transform(input)

In [12]:
embedding_shape = 32
batch_size = 128
epochs = 300

In [13]:
data_s, data_e = '2001/1/1', '2018/12/31'
target_s, target_e = '2019/1/1', '2019/12/31'
feature_days = 10

In [14]:
sne_dict = {
    'data':{},
    'tar':{}
}
embed_dict = {
    'data':{},
    'tar':{}
}

In [15]:
ts_id = 2
x, y, data_result = build_XY(data_s, data_e, 
                                 gen_data_config['selected_features'], 
                                 feature_days, ts_id)
tar_x, tar_y, tar_result = build_XY(target_s, target_e, 
                                 gen_data_config['selected_features'], 
                                 feature_days, ts_id)

In [16]:
'''for t-SNE'''
feature_day = 1

sne_data, _, _ = build_XY(data_s, data_e, 
                                 ['Open', 'Close'], 
                                 feature_day, 2)
sne_tar, _, _ = build_XY(target_s, target_e, 
                                 ['Open', 'Close'], 
                                 feature_day, 2)
sne_tr_y = []
sne_te_y = []
# data
for r in sne_data:
    if(r[1] > r[0]):
        sne_tr_y.append(0)
    else:
        sne_tr_y.append(1)
# target
for r in sne_tar:
    if(r[1] > r[0]):
        sne_te_y.append(0)
    else:
        sne_te_y.append(1)

In [17]:
print(f'x的shape：{x.shape}')
print(f"一天的k-bar vector長度：{len(gen_data_config['selected_features'])}")

x的shape：(4456, 430)
一天的k-bar vector長度：43


# Task_1

給予t, t-1, t-2的資料作為input預測t-1, t-2

Loss:min(t-1,t-2的RMSE)

In [18]:
task = '1'

In [19]:
'''Data'''
locals()[f'task_{task}_y'] = x[:, len(gen_data_config['selected_features']):]
x = np.reshape(x, (-1, feature_days, len(gen_data_config['selected_features'])))
tar_x = np.reshape(tar_x, (-1, feature_days, len(gen_data_config['selected_features'])))

In [20]:
'''build model'''
model = build_embed_model('rnn', 
                           x.shape, 
                           locals()[f'task_{task}_y'].shape, 
                           embedding_shape, 
                          'sigmoid')
sgd = SGD(lr=0.1, momentum=0.1, decay=0.0, nesterov=False)
model.compile(loss='mean_squared_error',
                  optimizer=sgd,
                  metrics=['accuracy'])

In [21]:
'''Fit models and use validation_split=0.1 '''
history = model.fit(x, 
                    locals()[f'task_{task}_y'],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True,
                    validation_split=0.1)

Train on 4010 samples, validate on 446 samples
Epoch 1/300
4010/4010 [==============================] - 4s 920us/step - loss: 12766.0782 - accuracy: 0.0589 - val_loss: 9533.9007 - val_accuracy: 0.1973
Epoch 2/300
4010/4010 [==============================] - 3s 666us/step - loss: 12754.0083 - accuracy: 0.0863 - val_loss: 9529.1089 - val_accuracy: 0.3049
Epoch 3/300
4010/4010 [==============================] - 3s 686us/step - loss: 12750.6782 - accuracy: 0.1072 - val_loss: 9527.4262 - val_accuracy: 0.3117
Epoch 4/300
4010/4010 [==============================] - 3s 675us/step - loss: 12749.4042 - accuracy: 0.1187 - val_loss: 9526.6392 - val_accuracy: 0.2937
Epoch 5/300
4010/4010 [==============================] - 3s 682us/step - loss: 12748.6101 - accuracy: 0.1192 - val_loss: 9525.5344 - val_accuracy: 0.2534
Epoch 6/300
4010/4010 [==============================] - 3s 668us/step - loss: 12747.8808 - accuracy: 0.1127 - val_loss: 9525.5613 - val_accuracy: 0.2018
Epoch 7/300
4010/4010 [======

4010/4010 [==============================] - 3s 666us/step - loss: 12742.6238 - accuracy: 0.0297 - val_loss: 9519.9184 - val_accuracy: 0.0112
Epoch 54/300
4010/4010 [==============================] - 3s 677us/step - loss: 12742.6686 - accuracy: 0.0317 - val_loss: 9519.8798 - val_accuracy: 0.0224
Epoch 55/300
4010/4010 [==============================] - 3s 671us/step - loss: 12742.5911 - accuracy: 0.0309 - val_loss: 9520.0648 - val_accuracy: 0.0112
Epoch 56/300
4010/4010 [==============================] - 3s 669us/step - loss: 12742.5710 - accuracy: 0.0342 - val_loss: 9519.9749 - val_accuracy: 0.0224
Epoch 57/300
4010/4010 [==============================] - 3s 672us/step - loss: 12742.5257 - accuracy: 0.0329 - val_loss: 9520.1401 - val_accuracy: 0.0135
Epoch 58/300
4010/4010 [==============================] - 3s 673us/step - loss: 12742.5200 - accuracy: 0.0289 - val_loss: 9519.7438 - val_accuracy: 0.0090
Epoch 59/300
4010/4010 [==============================] - 3s 675us/step - loss: 127

Epoch 106/300
4010/4010 [==============================] - 3s 678us/step - loss: 12741.5681 - accuracy: 0.0279 - val_loss: 9518.9837 - val_accuracy: 0.0247
Epoch 107/300
4010/4010 [==============================] - 3s 687us/step - loss: 12741.5996 - accuracy: 0.0247 - val_loss: 9518.8401 - val_accuracy: 0.0179
Epoch 108/300
4010/4010 [==============================] - 3s 675us/step - loss: 12741.5572 - accuracy: 0.0247 - val_loss: 9518.7459 - val_accuracy: 0.0112
Epoch 109/300
4010/4010 [==============================] - 3s 679us/step - loss: 12741.5076 - accuracy: 0.0234 - val_loss: 9518.9138 - val_accuracy: 0.0090
Epoch 110/300
4010/4010 [==============================] - 3s 675us/step - loss: 12741.4955 - accuracy: 0.0229 - val_loss: 9518.8427 - val_accuracy: 0.0179
Epoch 111/300
4010/4010 [==============================] - 3s 671us/step - loss: 12741.5076 - accuracy: 0.0242 - val_loss: 9518.6793 - val_accuracy: 0.0135
Epoch 112/300
4010/4010 [==============================] - 3s 67

4010/4010 [==============================] - 3s 659us/step - loss: 12741.1011 - accuracy: 0.0234 - val_loss: 9518.6171 - val_accuracy: 0.0112
Epoch 159/300
4010/4010 [==============================] - 3s 678us/step - loss: 12741.1346 - accuracy: 0.0219 - val_loss: 9518.4549 - val_accuracy: 0.0179
Epoch 160/300
4010/4010 [==============================] - 3s 679us/step - loss: 12741.1048 - accuracy: 0.0214 - val_loss: 9518.4371 - val_accuracy: 0.0179
Epoch 161/300
4010/4010 [==============================] - 3s 677us/step - loss: 12741.0663 - accuracy: 0.0224 - val_loss: 9518.5057 - val_accuracy: 0.0202
Epoch 162/300
4010/4010 [==============================] - 3s 678us/step - loss: 12741.0962 - accuracy: 0.0200 - val_loss: 9518.3755 - val_accuracy: 0.0179
Epoch 163/300
4010/4010 [==============================] - 3s 667us/step - loss: 12741.0901 - accuracy: 0.0214 - val_loss: 9518.3532 - val_accuracy: 0.0202
Epoch 164/300
4010/4010 [==============================] - 3s 679us/step - los

4010/4010 [==============================] - 3s 676us/step - loss: 12740.7755 - accuracy: 0.0192 - val_loss: 9518.1770 - val_accuracy: 0.0202
Epoch 211/300
4010/4010 [==============================] - 3s 679us/step - loss: 12740.7798 - accuracy: 0.0202 - val_loss: 9518.3105 - val_accuracy: 0.0112
Epoch 212/300
4010/4010 [==============================] - 3s 755us/step - loss: 12740.7982 - accuracy: 0.0190 - val_loss: 9518.3970 - val_accuracy: 0.0157
Epoch 213/300
4010/4010 [==============================] - 3s 739us/step - loss: 12740.7855 - accuracy: 0.0152 - val_loss: 9518.2409 - val_accuracy: 0.0135
Epoch 214/300
4010/4010 [==============================] - 4s 1ms/step - loss: 12740.7661 - accuracy: 0.0170 - val_loss: 9518.2728 - val_accuracy: 0.0135
Epoch 215/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.7590 - accuracy: 0.0185 - val_loss: 9518.1854 - val_accuracy: 0.0179
Epoch 216/300
4010/4010 [==============================] - 5s 1ms/step - loss: 127

4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5535 - accuracy: 0.0135 - val_loss: 9517.9989 - val_accuracy: 0.0135
Epoch 264/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5509 - accuracy: 0.0145 - val_loss: 9518.0643 - val_accuracy: 0.0179
Epoch 265/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5590 - accuracy: 0.0147 - val_loss: 9517.9581 - val_accuracy: 0.0135
Epoch 266/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5236 - accuracy: 0.0137 - val_loss: 9518.0840 - val_accuracy: 0.0112
Epoch 267/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5268 - accuracy: 0.0142 - val_loss: 9518.0103 - val_accuracy: 0.0157
Epoch 268/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5217 - accuracy: 0.0147 - val_loss: 9518.0328 - val_accuracy: 0.0157
Epoch 269/300
4010/4010 [==============================] - 5s 1ms/step - loss: 12740.5247 

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 256)           307200    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dense_2 (Dense)              (None, 387)              

In [23]:
'''get embedding output'''
'''get embedding output'''
get_layer_output = keras.backend.function([model.layers[0].input],
                                  [model.layers[5].output])
embed_dict['data'][task] = get_layer_output([x])[0]
embed_dict['tar'][task] = get_layer_output([tar_x])[0]

In [24]:
'''data t-SNE'''
sne_dict['data'][task]  = tsne_trans(embed_dict['data'][task])
sne_dict['tar'][task] = tsne_trans(embed_dict['tar'][task])

# Task_2a
分類 t+1 是上漲或下跌

In [25]:
task = '2a'

In [26]:
feature_day = 0
task_2a, _, _ = build_XY(data_s, data_e, 
                                 ['Open', 'Close'], 
                             feature_day, ts_id)

task_2a_y = []
for r in task_2a:
    if(r[1] > r[0]):
        task_2a_y.append(0)
    else:
        task_2a_y.append(1)
task_2a_y = keras.utils.to_categorical(task_2a_y, 2)

TypeError: build_XY() missing 1 required positional argument: 'ts_id'

In [ ]:
'''build model'''
model = build_embed_model('rnn', x.shape, task_2a_y.shape, 
                                 embedding_shape, 'softmax')
model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])

In [ ]:
'''Fit models and use validation_split=0.1 '''
history = model.fit(x, 
                    task_2a_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True,
                    validation_split=0.1)

In [ ]:
'''get embedding output'''
get_layer_output = keras.backend.function([model.layers[0].input],
                                  [model.layers[5].output])
embed_dict['data'][task] = get_layer_output([x])[0]
embed_dict['tar'][task] = get_layer_output([tar_x])[0]

In [ ]:
'''data t-SNE'''
sne_dict['data'][task]  = tsne_trans(embed_dict['data'][task])
sne_dict['tar'][task] = tsne_trans(embed_dict['tar'][task])

# Task_2b
分類 t 上漲或下跌

In [ ]:
task = '2b'

In [ ]:
feature_day = 1
task_2b, _, _ = build_XY(data_s, data_e, 
                                 ['Open', 'Close'], 
                             feature_day)

task_2b_y = []
for r in task_2a:
    if(r[1] > r[0]):
        task_2b_y.append(0)
    else:
        task_2b_y.append(1)
task_2b_y = keras.utils.to_categorical(task_2b_y, 2)

In [ ]:
'''build model'''
model = build_embed_model('rnn', x.shape, task_2b_y.shape, 
                                 embedding_shape, 'softmax')
model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])

In [ ]:
'''Fit models and use validation_split=0.1 '''
history = model.fit(x, 
                    task_2b_y,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True,
                    validation_split=0.1)

In [ ]:
'''get embedding output'''
get_layer_output = keras.backend.function([model.layers[0].input],
                                  [model.layers[5].output])
embed_dict['data'][task] = get_layer_output([x])[0]
embed_dict['tar'][task] = get_layer_output([tar_x])[0]

In [ ]:
'''data t-SNE'''
sne_dict['data'][task]  = tsne_trans(embed_dict['data'][task])
sne_dict['tar'][task] = tsne_trans(embed_dict['tar'][task])

# t-SNE

In [ ]:
fig = plt.figure(figsize=(30, 20))
ax1 = fig.add_subplot(321)
ax2 = fig.add_subplot(322)
ax3 = fig.add_subplot(323)
ax4 = fig.add_subplot(324)
ax5 = fig.add_subplot(325)
ax6 = fig.add_subplot(326)

for i, t in zip(range(1,4), ['1', '2a', '2b']):
    i = i*2
    left, right = i-1, i
    '''plot left'''
    X_tsne = sne_dict['data'][t]
    x_min, x_max = X_tsne.min(0), X_tsne.max(0)
    X_norm = (X_tsne - x_min) / (x_max - x_min)  # 正規化
    for i in range(X_norm.shape[0]):
        locals()[f'ax{left}'].text(X_norm[i, 0], X_norm[i, 1], str(sne_tr_y[i]), color=plt.cm.Set1(sne_tr_y[i]), 
             fontdict={'weight': 'bold', 'size': 9})
    '''plot right'''
    X_tsne = sne_dict['tar'][t]
    x_min, x_max = X_tsne.min(0), X_tsne.max(0)
    X_norm = (X_tsne - x_min) / (x_max - x_min)  # 正規化
    for i in range(X_norm.shape[0]):
        locals()[f'ax{right}'].text(X_norm[i, 0], X_norm[i, 1], str(sne_te_y[i]), color=plt.cm.Set1(sne_te_y[i]), 
             fontdict={'weight': 'bold', 'size': 9})

    locals()[f'ax{left}'].set_title('train', fontsize=20)
    locals()[f'ax{right}'].set_title('test', fontsize=20)

fig.suptitle(f't-SNE model:LSTM_{embedding_shape}', fontsize=30)

# save

In [ ]:
from pickle import dump
fig.savefig(f'./fig/tsne_rnn_{embedding_shape}_{feature_days}.png')
file = open(f'./embedding_data/rnn_{embedding_shape}_{feature_days}', 'wb')
dump(embed_dict, file)